In [3]:
import torch
from torchsummary import summary
from layers.CrossModalFusion import MultiModalAttentionGraph, ModalFusion

class Configs:
    def __init__(self):
        self.seq_len = 168
        self.top_k = 5
        self.num_nodes = 34
        self.num_channels = 8
        self.embed_dim = 256
        self.pred_len = 48

configs = Configs()

block_a = MultiModalAttentionGraph(34, 8, 64)
block_b = ModalFusion(42)


summary(block_a, [(34, 64), (8, 64)], device='cpu')
summary(block_b, [(2, 34, 64), (2,8, 64), (42, 42)], device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 34, 64]           4,160
            Linear-2               [-1, 34, 64]           4,160
            Linear-3                [-1, 8, 64]           4,160
            Linear-4                [-1, 8, 64]           4,160
Total params: 16,640
Trainable params: 16,640
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 4.25
Forward/backward pass size (MB): 0.04
Params size (MB): 0.06
Estimated Total Size (MB): 4.35
----------------------------------------------------------------


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 34 but got size 8 for tensor number 1 in the list.

In [5]:
import torch
from torchsummary import summary
from layers.BiFPN import BiStreamFPN

class Configs:
    def __init__(self):
        self.seq_len = 168
        self.top_k = 5
        self.num_nodes = 34
        self.num_channels = 8
        self.embed_dim = 256
        self.pred_len = 48

configs = Configs()

block_a = BiStreamFPN(8)



summary(block_a, (1, 8, 168, 192), device='cpu')


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 8, 112, 112]           3,136
       BatchNorm2d-2          [-1, 8, 112, 112]              16
            Conv2d-3          [-1, 8, 112, 112]             576
       BatchNorm2d-4          [-1, 8, 112, 112]              16
            Conv2d-5          [-1, 8, 112, 112]             576
       BatchNorm2d-6          [-1, 8, 112, 112]              16
        BasicBlock-7          [-1, 8, 112, 112]               0
            Conv2d-8           [-1, 16, 56, 56]           1,152
       BatchNorm2d-9           [-1, 16, 56, 56]              32
           Conv2d-10           [-1, 16, 56, 56]           2,304
      BatchNorm2d-11           [-1, 16, 56, 56]              32
           Conv2d-12           [-1, 16, 56, 56]             128
      BatchNorm2d-13           [-1, 16, 56, 56]              32
       BasicBlock-14           [-1, 16,

In [8]:
import torch
from torchsummary import summary
from layers.Spatial_Block import GraphBlock

class Configs:
    def __init__(self):
        self.seq_len = 168
        self.top_k = 5
        self.num_nodes = 34
        self.num_channels = 8
        self.embed_dim = 256
        self.pred_len = 48

configs = Configs()

block_a = GraphBlock(34)

summary(block_a, (34, 8, 12), device='cpu')


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
             nconv-1            [-1, 34, 8, 12]               0
             nconv-2            [-1, 34, 8, 12]               0
            Conv2d-3            [-1, 34, 8, 12]           3,502
            linear-4            [-1, 34, 8, 12]               0
           mixprop-5            [-1, 34, 8, 12]               0
              GELU-6            [-1, 34, 8, 12]               0
         LayerNorm-7               [-1, 96, 34]              68
Total params: 3,570
Trainable params: 3,570
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.17
Params size (MB): 0.01
Estimated Total Size (MB): 0.20
----------------------------------------------------------------


In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiScaleConc(nn.Module):

    def __init__(self, in_channels=3):
        super(MultiScaleConc, self).__init__()
        channels = 8  # define channels before using it
        self.start_conv = nn.Conv2d(in_channels, channels, kernel_size=7, stride=2, padding=3, bias=False)

        self.conv = nn.Conv2d(in_channels, 1* in_channels, kernel_size=1)
        self.bn = nn.BatchNorm2d(1* channels)
        
        self.scale_conv1 = nn.Conv2d(channels, 1* channels, kernel_size=1, stride=1)
        self.scale_conv2 = nn.Conv2d(channels, 2* channels, kernel_size=2, stride=2)
        self.scale_conv3 = nn.Conv2d(channels, 4* channels, kernel_size=4, stride=4)
        self.scale_conv4 = nn.Conv2d(channels, 8* channels, kernel_size=8, stride=8)
        

    def padding(self, x):
        # 计算需要填充的像素数
        padding_height = 224 - x.size(3)
        padding_width = 224 - x.size(4)
        # 计算在每个方向上需要填充的像素数
        padding_top = padding_height // 2
        padding_bottom = padding_height - padding_top
        padding_left = padding_width // 2
        padding_right = padding_width - padding_left
        # 使用pad函数填充图像
        x = F.pad(x, (padding_left, padding_right, padding_top, padding_bottom), mode='constant', value=0)
        return x

    def forward(self, x):
        x = torch.squeeze(self.padding(x))

        x = self.bn(self.start_conv(x))
        
        c1 = self.scale_conv1(x)
        c2 = self.scale_conv2(x)
        c3 = self.scale_conv3(x)
        c4 = self.scale_conv4(x)
        
        return [c1, c2, c3, c4]

model = MultiScaleConc()

input = torch.randn(32, 1, 3, 169, 192)

output = model(input)

print(output[0].shape)
print(output[1].shape)
print(output[2].shape)
print(output[3].shape)


torch.Size([32, 8, 112, 112])
torch.Size([32, 16, 56, 56])
torch.Size([32, 32, 28, 28])
torch.Size([32, 64, 14, 14])


In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F

conv1 = nn.Conv2d(8, 8, kernel_size=8, stride=8)
start_conv = nn.Conv2d(8, 8, kernel_size=7, stride=2, padding=3, bias=False)

a = torch.randn(32, 8, 224, 224)

b = start_conv(a)

print(b.shape)

torch.Size([32, 8, 112, 112])
